# Python for Genomic Data Analysis

## Final project: 

## Management and visualization of genetic sequences from whole genomes

The objectives of this project are: 

* 1. Create a function to extract genes of interest from one or more genomes. 
* 2. Create a function to filter the sequences to the desired bp length. 
* 3. Align the retrived sequences with mafft. 
* 4. Generate a gene tree and visualize it.

The gene of interest is the ATPase alpha subunit. It is important in the process of detoxification performed by insects against toxic substances of the plants they consume. Here we will analyze eighteen genomes of milkweed longhorn beetles and other insect orders including butterflies and true bugs.

## Installing required software

BioPython is already installed so let's confirm we are in the bioinfo environment:

In [9]:
!echo $CONDA_DEFAULT_ENV

bioinfo


Load libraries and install iqtree and ete.

In [271]:
from Bio import SeqIO, AlignIO
from Bio.SeqRecord import SeqRecord
from ete3 import Tree, TreeStyle, NodeStyle

# conda install -c bioconda iqtree ete3

See content of current directory.

In [33]:
!ls -lh #$WD

total 6568
-rw-r--r--@  1 nayeligutierrez  staff    30K Feb  1 11:51 1.assembly_supernova.ipynb
-rw-r--r--@  1 nayeligutierrez  staff    31K Mar  3 15:52 2.assembly_optimization.ipynb
-rw-r--r--@  1 nayeligutierrez  staff   203K Jan 28 14:53 3.repeatmasker.ipynb
-rw-r--r--@  1 nayeligutierrez  staff    12K Jan 13 18:33 4.busco_install(unsuccesfull1).ipynb
-rw-r--r--@  1 nayeligutierrez  staff    18K Jan 13 18:33 5.busco_install(unsuccesfull2).ipynb
-rw-r--r--@  1 nayeligutierrez  staff    23K Apr 28 19:58 6.busco_install(succesfull).ipynb
-rw-r--r--@  1 nayeligutierrez  staff    15K Jan 14 20:49 7.illumina_qc.ipynb
-rw-r--r--@  1 nayeligutierrez  staff    49K Feb 24 14:21 8.assembly_illumina.ipynb
-rw-r--r--@  1 nayeligutierrez  staff    60K Apr 30 19:26 9.1.phylogeny.ipynb
-rw-r--r--@  1 nayeligutierrez  staff    34K Apr 30 19:26 9.2.Maker_Braker.ipynb
-rw-r--r--@  1 nayeligutierrez  staff   158K Apr 28 18:28 9.3.Blast.ipynb
-rw-r--r--@  1 nayeligutierrez  staff    14K May  3 18:22 9.

Looks like we are in the right directory, now let's create a variable with the directory where the genome files are located:

In [34]:
genomes_dir = 'functionally_annotated_genomes/'
!ls -l $genomes_dir

total 929936
-rw-r--r--@ 1 nayeligutierrez  staff  42564506 Apr 28 19:47 Aglabripennis.fasta
-rw-r--r--@ 1 nayeligutierrez  staff  26454908 Apr 29 18:33 Bmori.fasta
-rw-r--r--  1 nayeligutierrez  staff  18140062 May  4 22:59 Clectularius.fasta
-rw-r--r--@ 1 nayeligutierrez  staff  22515159 May  3 23:17 Dplexippus.fasta
-rw-r--r--@ 1 nayeligutierrez  staff  23212105 May  3 21:19 Ofasciatus.fasta
-rw-r--r--@ 1 nayeligutierrez  staff  18863127 Mar 19 22:58 Rlineaticollis1.fasta
-rw-r--r--@ 1 nayeligutierrez  staff  27303125 Mar 27 19:21 Tannulatus1.fasta
-rw-r--r--@ 1 nayeligutierrez  staff  23329748 Mar 12 16:07 Tannulatus2.fasta
-rw-r--r--@ 1 nayeligutierrez  staff  52399286 Apr 28 20:56 Tcastaneum.fasta
-rw-r--r--@ 1 nayeligutierrez  staff  24256983 Mar 12 23:04 Tdiscoideus1.fasta
-rw-r--r--@ 1 nayeligutierrez  staff  30193583 Mar 17 23:54 Tdiscoideus2.fasta
-rw-r--r--@ 1 nayeligutierrez  staff  23528473 Mar 14 20:39 Tfemoratus1.fasta
-rw-r--r--@ 1 nayeligutierrez  staff  36986249 Mar 

## 1. Extract gene secuences from genome files

We have 18 genome annotation files from where I want to extract only the sequences that correspond to ATPase subunit alpha. To avoid doing it one by one, let's create a function that uses as argument the name of the gene we are looking for and the directory where the genomes are located. The function will:

**a)** Extract sequences that match the desired gene.

**b)** Append sequences to a file.

**c)** Rename sequences to indicate what species they correspond to.

In [94]:
def genefind(gene_name, genomes_dir, split_name=True, ignore_case=True):
  """
  Looks for genes one or several genomes. The sequences found are renamed using the file name where they were 
  found and are appended to a file. 
  param gene_name is name of the gene of interest.
  param genomes_dir is directory where the genome(s) are.
  param split_name looks for the name of the gene with its letters arranged in all order combinations.
  param ignore_case makes gene_name lowercase.
  """
  # Split the gene name because it can be in a different order (default)
  if split_name:
    terms = gene_name.split() # "ATPase subunit Alpha" --> ["ATPase","subunit", "Alpha"]
  else:
    terms = [gene_name]
  # Make all terms lowercase if ignoring case (default)
  if ignore_case:
    terms = [term.lower() for term in terms] # --> ["atpase","subunit", "alpha"]
  
  # Define variables
  genomes_files = 'genomes_files.list'
  !ls $genomes_dir > $genomes_files
  !cat $genomes_files

  gene_name_file = gene_name.replace(" ", "_") + ".fasta"

  # Clear the file so that it starts empty
  with open(gene_name_file, 'w') as f:
    pass

  with open(genomes_files, 'r') as f:
    for filename in f:
      # Remove space between names
      filename = filename.strip()
      print("\n" + filename)
      seqrecs = SeqIO.parse(genomes_dir + filename, 'fasta') 

      # Go over each record in this file
      for rec in seqrecs:
        # Create variable for the description of each record
        desc = rec.description
        if ignore_case:
          desc = desc.lower()
        # Extract sequences that match the gene
        if all([term in desc for term in terms]):
          # Append sequences to a file where all sequences will be stored
          print(rec.description)
          with open(gene_name_file, 'a') as g:   
            # Rename sequences to indicate what species they correspond to
            rec.id = filename.replace('.fasta', '_') + rec.id
            rec.description = ''
            g.write(rec.format('fasta'))
  # Count the number of sequences found                      
  number_sequences = !grep -c "^>" $gene_name_file
  # Redefine data type of the variable (from string to integer)
  number_sequences = int(number_sequences[0])
  print(f"\n{number_sequences} sequences matching \"{gene_name}\" have been retrieved in the file \"{gene_name_file}\"")

Let's use genefind() to look for the sequences that match the name of the gene ATPase subunit alpha. The function is designed to find sequences that match that name even when the words are not in the right order. For example, it will find sequences named "ATPase subunit alpha" as well as those named "ATPase alpha subunit" and "subunit alpha ATPase", etc.: 

In [95]:
genefind("ATPase subunit alpha", "functionally_annotated_genomes/")

Aglabripennis.fasta
Bmori.fasta
Clectularius.fasta
Dplexippus.fasta
Ofasciatus.fasta
Rlineaticollis1.fasta
Tannulatus1.fasta
Tannulatus2.fasta
Tcastaneum.fasta
Tdiscoideus1.fasta
Tdiscoideus2.fasta
Tfemoratus1.fasta
Tfemoratus2.fasta
Tpilosus1.fasta
Tpilosus2.fasta
Tskillmani1.fasta
Ttetrophthalmus.fasta

Aglabripennis.fasta
lcl|NW_019416354.1_cds_XP_018565491.1_5686 [gene=LOC108906661] [db_xref=GeneID:108906661] [protein=sodium/potassium-transporting ATPase subunit alpha] [protein_id=XP_018565491.1] [location=complement(join(259262..259373,259425..259539,260985..261173,261524..261769,261830..262102,262161..262422,264862..265245,265508..265731,265783..266028,267310..267546,268306..268435,269663..269830,270039..270211,297296..297445,341693..341813))] [gbkey=CDS]
lcl|NW_019416398.1_cds_XP_018562049.1_6847 [gene=LOC108904109] [db_xref=GeneID:108904109] [protein=sodium/potassium-transporting ATPase subunit alpha isoform X1] [protein_id=XP_018562049.1] [location=join(996763..996792,1049362.

6644992.g15 sodium/potassium-transporting ATPase subunit alpha isoform X6
8445445.g35 sodium/potassium-transporting ATPase subunit alpha isoform X9
9280121.g26 sodium/potassium-transporting ATPase subunit alpha isoform X3
12447157.g16 sodium/potassium-transporting ATPase subunit alpha
15155240.g8 sodium/potassium-transporting ATPase subunit alpha isoform X4
16086664.g19 sodium/potassium-transporting ATPase subunit alpha-like
17167391.g29 Sodium/potassium-transporting ATPase subunit alpha
17373146.g32 sodium/potassium ATPase alpha subunit
20169353.g84 Na+/K+ ATPase alpha subunit
20207899.g86 sodium/potassium-transporting ATPase subunit alpha
20254541.g134 sodium/potassium-transporting ATPase subunit alpha isoform X6
20254542.g135 sodium/potassium-transporting ATPase subunit alpha isoform X6

Tannulatus1.fasta
2020233.g45 sodium/potassium-transporting ATPase subunit alpha-like
2465216.g23 sodium/potassium-transporting ATPase subunit alpha isoform X2
4949613.g52 sodium/potassium-transport

2086639.g45 sodium/potassium-transporting ATPase subunit alpha isoform X3
5565729.g65 sodium/potassium-transporting ATPase subunit alpha-like
6702885.g23 sodium/potassium-transporting ATPase subunit alpha-like
6709994.g48 sodium/potassium-transporting ATPase subunit alpha-like
6770603.g14 Na+/K+ ATPase alpha subunit
6795269.g101 sodium/potassium-transporting ATPase subunit alpha isoform X6

Tdiscoideus2.fasta
2890578.g52 Na+/K+ ATPase alpha subunit
3689124.g70 sodium/potassium-transporting ATPase subunit alpha-like
3746555.g55 Na+/K+ ATPase alpha subunit
4198478.g3 sodium/potassium-transporting ATPase subunit alpha-like
2469352.g65 Na+/K+ ATPase alpha subunit

Tfemoratus1.fasta
1654610.g24 sodium/potassium ATPase alpha subunit
2246706.g47 sodium/potassium ATPase alpha subunit
3077924.g10 Na+,K+ ATPase alpha-subunit 1
3178549.g18 sodium/potassium-transporting ATPase subunit alpha
9062756.g67 sodium/potassium-transporting ATPase subunit alpha-like
9193525.g41 sodium/potassium-transportin

Result: 123 sequences matching "ATPase subunit alpha" have been retrieved in the file "ATPase_subunit_alpha.fasta"

## 2. Filter the sequences to a desired bp length

Using genefind(), 123 sequences were retrieved. The length of the sequences varies from 3460 to 50 bp. Some of the sequences are very short to be included in further steps of the pipeline. Before moving forward with sequence alignment, we will use the seqfilter() function to fliter sequences shorter than 1000 bp (the minimum length chosen is an arbitrary):

In [96]:
def seqfilter(fasta_file, min_length):
  """
  Filter sequences with a user-defined threshold and creates two files, one containing sequences that have 
  passed the threshold and one with sequences that did not. 
  param fasta_file is name of the file.
  param min_length is the number of bp of the threshold.
  """
  # Create fasta files where to storage sequences 
  long_seqs_file = fasta_file.replace("fasta", "filtered.fasta")
  short_seqs_file = fasta_file.replace("fasta", "shortseqs.fasta")
    
  # Define seqrecs using SeqIO
  seqrecs = SeqIO.parse(fasta_file, "fasta") 
    
  # Clear the files so that they start empty
  with open(long_seqs_file, 'w') as f:
    pass
  with open(short_seqs_file, 'w') as f:
    pass     
  
  # Go over each record in the file
  for rec in seqrecs:
    # If the record is longer than the min_length
    if len(rec) > min_length:
      # Append sequences to a file where all long sequences will be stored
      print(rec.description)
      with open(long_seqs_file, 'a') as g:   
        g.write(rec.format('fasta'))
    else:
      # Append sequences to a file where all short sequences will be stored
      print(rec.description)
      with open(short_seqs_file, 'a') as g:   
        g.write(rec.format('fasta'))
  # Count the number of sequences found           
  num_long_sequences = !grep -c "^>" $long_seqs_file
  # Redefine data type of the variable (from string to integer)
  num_long_sequences = int(num_long_sequences[0])
  print(f"\n{num_long_sequences} sequences passed the {min_length} bp threshold. They have been retrieved in the file \"{long_seqs_file}\"")          

Let's use seqfilter() to filter the sequences in the "ATPase_subunit_alpha.fasta" file: 

In [97]:
seqfilter("ATPase_subunit_alpha.fasta", 1000)

Aglabripennis_lcl|NW_019416354.1_cds_XP_018565491.1_5686
Aglabripennis_lcl|NW_019416398.1_cds_XP_018562049.1_6847
Aglabripennis_lcl|NW_019416398.1_cds_XP_018562050.1_6848
Aglabripennis_lcl|NW_019416398.1_cds_XP_018562051.1_6849
Aglabripennis_lcl|NW_019416398.1_cds_XP_018562054.1_6850
Aglabripennis_lcl|NW_019416398.1_cds_XP_018562052.1_6851
Aglabripennis_lcl|NW_019416398.1_cds_XP_018562053.1_6852
Aglabripennis_lcl|NW_019416398.1_cds_XP_018562055.1_6853
Aglabripennis_lcl|NW_019416398.1_cds_XP_018562056.1_6854
Aglabripennis_lcl|NW_019416398.1_cds_XP_018562057.1_6855
Aglabripennis_lcl|NW_019417389.1_cds_XP_023312013.1_17588
Clectularius_CLEC006776-RA
Dplexippus_DPOGS214640-TA
Dplexippus_DPOGS213623-TA
Dplexippus_DPOGS211186-TA
Ofasciatus_lcl|KK854243.1_cds_PTY13626.1_4739
Ofasciatus_lcl|KK854243.1_cds_PTY13634.1_4747
Ofasciatus_lcl|KK854243.1_cds_PTY13635.1_4748
Ofasciatus_lcl|KK854249.1_cds_PTY13707.1_4820
Ofasciatus_lcl|KK854697.1_cds_PTY17974.1_9087
Ofasciatus_lcl|KK854787.1_cds_PTY1871

Result: only 70 of the 123 sequences were longer than 1000 bp. Sequences of less than 1000 bp were stored in a different file in case they need to be inspected later.

## 3. Align the sequences retrieved using mafft

Now that we have only sequences longer than 1000 bp, we'll proceed with aligning them.

In [14]:
# Create variable for aligned sequences
filtered_atpases_file = "ATPase_subunit_alpha.filtered.fasta"
# Create a file to storage aligned sequences
aligned_file = filtered_atpases_file.replace(".fasta", ".mafft.fasta")

# Use mafft to align the sequences
!mafft $filtered_atpases_file > $aligned_file

nthread = 0
nthreadpair = 0
nthreadtb = 0
ppenalty_ex = 0
stacksize: 8192 kb
generating a scoring matrix for nucleotide (dist=200) ... done
Gap Penalty = -1.53, +0.00, +0.00



Making a distance matrix ..

There are 31 ambiguous characters.
    1 / 70
done.

Constructing a UPGMA tree (efffree=0) ... 
   60 / 70
done.

Progressive alignment 1/2... 
STEP    55 / 69 
Reallocating..done. *alloclen = 7993
STEP    69 / 69 
done.

Making a distance matrix from msa.. 
    0 / 70
done.

Constructing a UPGMA tree (efffree=1) ... 
   60 / 70
done.

Progressive alignment 2/2... 
STEP    54 / 69 
Reallocating..done. *alloclen = 7951
STEP    69 / 69 
done.

disttbfast (nuc) Version 7.475
alg=A, model=DNA200 (2), 1.53 (4.59), -0.00 (-0.00), noshift, amax=0.0
0 thread(s)


Strategy:
 FFT-NS-2 (Fast but rough)
 Progressive method (guide trees were built 2 times.)

If unsure which option to use, try 'mafft --auto input > output'.
For more information, see 'mafft --help', 'mafft --man' and the mafft page

## 4. Generate a gene tree

The next step is to use the aligned sequences to generate a gene tree. For this step we will assume that all recovered sequences are descendants of a common ancestor and we will use a maximum likelihood algorithm to obtain a gene tree.

In [ ]:
# Use ModelFinderPlus (MFP) in IQTREE to determine the best-fit model:
!iqtree -s $aligned_file -m MFP

What model best fitted our data?

In [17]:
!cat ATPase_subunit_alpha.filtered.mafft.fasta.iqtree

IQ-TREE 2.1.2 COVID-edition built Mar 30 2021

Input file name: ATPase_subunit_alpha.filtered.mafft.fasta
Type of analysis: ModelFinder + tree reconstruction
Random seed number: 427365

REFERENCES
----------

To cite IQ-TREE please use:

Bui Quang Minh, Heiko A. Schmidt, Olga Chernomor, Dominik Schrempf,
Michael D. Woodhams, Arndt von Haeseler, and Robert Lanfear (2020)
IQ-TREE 2: New models and efficient methods for phylogenetic inference
in the genomic era. Mol. Biol. Evol., in press.
https://doi.org/10.1093/molbev/msaa015

To cite ModelFinder please use: 

Subha Kalyaanamoorthy, Bui Quang Minh, Thomas KF Wong, Arndt von Haeseler,
and Lars S Jermiin (2017) ModelFinder: Fast model selection for
accurate phylogenetic estimates. Nature Methods, 14:587–589.
https://doi.org/10.1038/nmeth.4285

SEQUENCE ALIGNMENT
------------------

Input data: 70 sequences with 4382 nucleotide sites
Number of constant sites: 1583 (= 36.1251% of all sites)
Number of invariant (constant or ambiguous constan

Looks like GTR+F+G4 is the best-fit model. Now let's look at the tree:

In [26]:
!cat ATPase_subunit_alpha.filtered.mafft.fasta.treefile.nwk

(Aglabripennis_lcl|NW_019416354.1_cds_XP_018565491.1_5686:0.1275601792,((((((((((((((((Aglabripennis_lcl|NW_019416398.1_cds_XP_018562049.1_6847:0.0000028573,(Aglabripennis_lcl|NW_019416398.1_cds_XP_018562054.1_6850:0.0024603637,(Aglabripennis_lcl|NW_019416398.1_cds_XP_018562052.1_6851:0.0000010000,Aglabripennis_lcl|NW_019416398.1_cds_XP_018562055.1_6853:0.0000010000):0.0010171896):0.0000021655):0.0042687197,(Aglabripennis_lcl|NW_019416398.1_cds_XP_018562050.1_6848:0.0000022748,(Aglabripennis_lcl|NW_019416398.1_cds_XP_018562053.1_6852:0.0000010000,Aglabripennis_lcl|NW_019416398.1_cds_XP_018562056.1_6854:0.0000010000):0.0010176657):0.0050679461):0.0062751523,Aglabripennis_lcl|NW_019416398.1_cds_XP_018562051.1_6849:0.0000021500):0.0000020568,(Aglabripennis_lcl|NW_019416398.1_cds_XP_018562057.1_6855:0.0017597029,Tfemoratus2_4327769.g89:8.9921462315):0.0000027158):0.0870280788,((((((Tannulatus1_2465216.g23:0.0005291897,Tpilosus2_4819112.g7:0.0015910004):0.0000010000,Tannulatus2_5311409.g40:

Looks good but not very useful to see the topology. The next thing to do is use ete to visualize the tree.

### Visualize the gene tree

We'll use ete to see the tree.

In [31]:
# Create a variable for the gene tree generated with IQTREE
tree_file = "ATPase_subunit_alpha.filtered.mafft.fasta.treefile.nwk"

# Assign the tree to the variable iqtree
iqtree = Tree(tree_file)

Let's take a look at the tree:

In [32]:
print(iqtree)


   /-Aglabripennis_lcl|NW_019416354.1_cds_XP_018565491.1_5686
  |
  |                                                /-Aglabripennis_lcl|NW_019416398.1_cds_XP_018562049.1_6847
  |                                             /-|
  |                                            |  |   /-Aglabripennis_lcl|NW_019416398.1_cds_XP_018562054.1_6850
  |                                            |   \-|
  |                                            |     |   /-Aglabripennis_lcl|NW_019416398.1_cds_XP_018562052.1_6851
  |                                          /-|      \-|
  |                                         |  |         \-Aglabripennis_lcl|NW_019416398.1_cds_XP_018562055.1_6853
  |                                         |  |
  |                                         |  |   /-Aglabripennis_lcl|NW_019416398.1_cds_XP_018562050.1_6848
  |                                       /-|   \-|
  |                                      |  |     |   /-Aglabripennis_lcl|NW_019416398.1_cds_XP_018562

So far the tree is unrooted. Let's root the tree to one of the sequences of *Danaus plexippus*, the monarch butterfly:

In [39]:
# Define a variable called Dplexippus

Dplexippus = "Dplexippus_DPOGS213623-TA"

# Define Dplexippus as the root of the tree
iqtree.set_outgroup(Dplexippus)
print(iqtree)


   /-Dplexippus_DPOGS213623-TA
  |
  |                                          /-Aglabripennis_lcl|NW_019416398.1_cds_XP_018562049.1_6847
  |                                       /-|
  |                                      |  |   /-Aglabripennis_lcl|NW_019416398.1_cds_XP_018562054.1_6850
  |                                      |   \-|
  |                                      |     |   /-Aglabripennis_lcl|NW_019416398.1_cds_XP_018562052.1_6851
  |                                    /-|      \-|
  |                                   |  |         \-Aglabripennis_lcl|NW_019416398.1_cds_XP_018562055.1_6853
  |                                   |  |
  |                                   |  |   /-Aglabripennis_lcl|NW_019416398.1_cds_XP_018562050.1_6848
  |                                 /-|   \-|
  |                                |  |     |   /-Aglabripennis_lcl|NW_019416398.1_cds_XP_018562053.1_6852
  |                                |  |      \-|
  |                                | 

### Create a nice-looking figure

We can get an idea of the topology by looking at the visualization created in the previous step. So far we see that some of the sequences are grouped by species, but there are others - especially in the genus *Tetraopes* (identifiable by a "T" before the species name) - where it would be interesting to take a closer look. To better visualize the topology we will color the tips (or leaves) according to their genus.

First let's print the leaf names:

In [43]:
for leaf in iqtree:
    print(leaf.name)

Dplexippus_DPOGS213623-TA
Aglabripennis_lcl|NW_019416398.1_cds_XP_018562049.1_6847
Aglabripennis_lcl|NW_019416398.1_cds_XP_018562054.1_6850
Aglabripennis_lcl|NW_019416398.1_cds_XP_018562052.1_6851
Aglabripennis_lcl|NW_019416398.1_cds_XP_018562055.1_6853
Aglabripennis_lcl|NW_019416398.1_cds_XP_018562050.1_6848
Aglabripennis_lcl|NW_019416398.1_cds_XP_018562053.1_6852
Aglabripennis_lcl|NW_019416398.1_cds_XP_018562056.1_6854
Aglabripennis_lcl|NW_019416398.1_cds_XP_018562051.1_6849
Aglabripennis_lcl|NW_019416398.1_cds_XP_018562057.1_6855
Tfemoratus2_4327769.g89
Tannulatus1_2465216.g23
Tpilosus2_4819112.g7
Tannulatus2_5311409.g40
Tpilosus1_4271307.g61
Ttetrophthalmus_maker-163393-exonerate_protein2genome-gene-0.5-mRNA-1
Tfemoratus1_3077924.g10
Tfemoratus2_3114834.g47
Tdiscoideus1_1854031.g58
Tdiscoideus2_2890578.g52
Tskillmani1_13332957.g20
Tcastaneum_lcl|NC_007422.5_cds_XP_015837163.1_15311
Tcastaneum_lcl|NC_007422.5_cds_XP_008196417.1_15312
Tcastaneum_lcl|NC_007422.5_cds_XP_008196422.1_153

Now we need to assign each species to a subgroup. We will create a dictionary where each leaf.name will be a key and the genus name will be its value:

In [254]:
# Create an empty dictionary called insect_group
insect_group = {}

# For each leave in the tree
for leaf in iqtree:
    # Evaluate its name
    if "Aglabripennis" in leaf.name:
    # and create a new set of key:value pair if it coitains the name of the genus
      insect_group[leaf.name] = 'Aglabripennis'
    elif "Tcastaneum" in leaf.name:
      insect_group[leaf.name] = 'Tcastaneum'
    elif "Dplexippus" in leaf.name:
      insect_group[leaf.name] = 'Dplexippus'
    elif "Ofasciatus" in leaf.name:
      insect_group[leaf.name] = 'Ofasciatus'
    elif "Tannulatus" in leaf.name:
      insect_group[leaf.name] = 'Tannulatus'
    elif "Tpilosus" in leaf.name:
      insect_group[leaf.name] = 'Tpilosus'
    elif "Tdiscoideus" in leaf.name:
      insect_group[leaf.name] = 'Tdiscoideus'
    elif "Tskillmani" in leaf.name:
      insect_group[leaf.name] = 'Tskillmani'
    elif "Tfemoratus" in leaf.name:
      insect_group[leaf.name] = 'Tfemoratus'
    elif "Ttetrophthalmus" in leaf.name:
      insect_group[leaf.name] = 'Ttetrophthalmus'
    elif "Clectularius" in leaf.name:
      insect_group[leaf.name] = 'Clectularius'
    else:
      break
# Print the key:value pairs  
for key, value in insect_group.items():
    print(key, ' : ', value)

Dplexippus_DPOGS213623-TA  :  Dplexippus
Aglabripennis_lcl|NW_019416398.1_cds_XP_018562049.1_6847  :  Aglabripennis
Aglabripennis_lcl|NW_019416398.1_cds_XP_018562054.1_6850  :  Aglabripennis
Aglabripennis_lcl|NW_019416398.1_cds_XP_018562052.1_6851  :  Aglabripennis
Aglabripennis_lcl|NW_019416398.1_cds_XP_018562055.1_6853  :  Aglabripennis
Aglabripennis_lcl|NW_019416398.1_cds_XP_018562050.1_6848  :  Aglabripennis
Aglabripennis_lcl|NW_019416398.1_cds_XP_018562053.1_6852  :  Aglabripennis
Aglabripennis_lcl|NW_019416398.1_cds_XP_018562056.1_6854  :  Aglabripennis
Aglabripennis_lcl|NW_019416398.1_cds_XP_018562051.1_6849  :  Aglabripennis
Aglabripennis_lcl|NW_019416398.1_cds_XP_018562057.1_6855  :  Aglabripennis
Tfemoratus2_4327769.g89  :  Tfemoratus
Tannulatus1_2465216.g23  :  Tannulatus
Tpilosus2_4819112.g7  :  Tpilosus
Tannulatus2_5311409.g40  :  Tannulatus
Tpilosus1_4271307.g61  :  Tpilosus
Ttetrophthalmus_maker-163393-exonerate_protein2genome-gene-0.5-mRNA-1  :  Ttetrophthalmus
Tfemorat

Result: It correctly assigned each leaf.name to the genus it corresponded.

BUT after having errors when trying to do:

    for leaf in iqtree:
        leaf.insect_gropu = insect_group[leaf.name]

I tried a different approach: use the output to create a dictionary by hand.

In [323]:
order = {
  'Dplexippus_DPOGS213623-TA'                                 :  "Dplexippus",
  'Aglabripennis_lcl|NW_019416398.1_cds_XP_018562049.1_6847'  :  "Aglabripennis",
  'Aglabripennis_lcl|NW_019416398.1_cds_XP_018562054.1_6850'  :  'Aglabripennis',
  'Aglabripennis_lcl|NW_019416398.1_cds_XP_018562052.1_6851'  :  "Aglabripennis",
  'Aglabripennis_lcl|NW_019416398.1_cds_XP_018562055.1_6853'  :  "Aglabripennis",
  'Aglabripennis_lcl|NW_019416398.1_cds_XP_018562050.1_6848'  :  "Aglabripennis",
  'Aglabripennis_lcl|NW_019416398.1_cds_XP_018562053.1_6852'  :  "Aglabripennis",
  'Aglabripennis_lcl|NW_019416398.1_cds_XP_018562056.1_6854'  :  "Aglabripennis",
  'Aglabripennis_lcl|NW_019416398.1_cds_XP_018562051.1_6849'  :  "Aglabripennis",
  'Aglabripennis_lcl|NW_019416398.1_cds_XP_018562057.1_6855'  :  "Aglabripennis",
  'Tfemoratus2_4327769.g89'                                   :  "Tfemoratus",
  'Tannulatus1_2465216.g23'                                   :  "Tannulatus",
  'Tpilosus2_4819112.g7'                                      :  "Tpilosus",
  'Tannulatus2_5311409.g40'                                   :  "Tannulatus",
  'Tpilosus1_4271307.g61'                                     :  "Tpilosus",
  'Ttetrophthalmus_maker-163393-exonerate_protein2genome-gene-0.5-mRNA-1'  :  "Ttetrophthalmus",
  'Tfemoratus1_3077924.g10'                                   :  "Tfemoratus",
  'Tfemoratus2_3114834.g47'                                   :  "Tfemoratus",
  'Tdiscoideus1_1854031.g58'                                  :  "Tdiscoideus",
  'Tdiscoideus2_2890578.g52'                                  :  "Tdiscoideus",
  'Tskillmani1_13332957.g20'                                  :  "Tskillmani",
  'Tcastaneum_lcl|NC_007422.5_cds_XP_015837163.1_15311'       :  "Tcastaneum",
  'Tcastaneum_lcl|NC_007422.5_cds_XP_008196417.1_15312'       :  "Tcastaneum",
  'Tcastaneum_lcl|NC_007422.5_cds_XP_008196422.1_15315'       :  "Tcastaneum",
  'Tcastaneum_lcl|NC_007422.5_cds_XP_015837164.1_15316'       :  "Tcastaneum",
  'Tcastaneum_lcl|NC_007422.5_cds_XP_008196420.1_15317'       :  "Tcastaneum",
  'Tcastaneum_lcl|NC_007422.5_cds_XP_008196423.1_15319'       :  "Tcastaneum",
  'Tcastaneum_lcl|NC_007422.5_cds_XP_008196418.1_15313'       :  "Tcastaneum",
  'Tcastaneum_lcl|NC_007422.5_cds_XP_008196421.1_15318'       :  "Tcastaneum",
  'Tcastaneum_lcl|NC_007422.5_cds_XP_008196424.1_15320'       :  "Tcastaneum",
  'Tcastaneum_lcl|NC_007422.5_cds_XP_008196419.1_15314'       :  "Tcastaneum",
  'Tcastaneum_lcl|NC_007422.5_cds_XP_008196425.1_15321'       :  "Tcastaneum",
  'Dplexippus_DPOGS214640-TA'                                 :  "Dplexippus",
  'Clectularius_CLEC006776-RA'                                :  "Clectularius",
  'Ofasciatus_lcl|KK854249.1_cds_PTY13707.1_4820'             :  "Ofasciatus",
  'Ofasciatus_lcl|KK854243.1_cds_PTY13634.1_4747'             :  "Ofasciatus",
  'Ofasciatus_lcl|KK854243.1_cds_PTY13635.1_4748'             :  "Ofasciatus",
  'Ofasciatus_lcl|KK854243.1_cds_PTY13626.1_4739'             :  "Ofasciatus",
  'Ofasciatus_lcl|KK854697.1_cds_PTY17974.1_9087'             :  "Ofasciatus",
  'Ofasciatus_lcl|KK854787.1_cds_PTY18716.1_9829'             :  "Ofasciatus",
  'Aglabripennis_lcl|NW_019417389.1_cds_XP_023312013.1_17588' :  "Aglabripennis",
  'Tannulatus1_5852914.g76'                                   :  "Tannulatus",
  'Tannulatus2_6685946.g8'                                    :  "Tannulatus",
  'Tpilosus1_5418222.g21'                                     :  "Tpilosus",
  'Tpilosus2_7357420.g51'                                     :  "Tpilosus",
  'Tpilosus1_5497471.g120'                                    :  "Tpilosus",
  'Tdiscoideus1_6770603.g14'                                  :  "Tdiscoideus",
  'Tdiscoideus2_3746555.g55'                                  :  "Tdiscoideus",
  'Tskillmani1_12903966.g5'                                   :  "Tskillmani",
  'Tfemoratus1_9193525.g41'                                   : "Tfemoratus",
  'Tfemoratus2_584367.g107'                                   :  "Tfemoratus",
  'Ttetrophthalmus_maker-158804-exonerate_protein2genome-gene-0.0-mRNA-1'  :  "Ttetrophthalmus",
  'Tdiscoideus2_3689124.g70'                                  :  "Tdiscoideus",
  'Tcastaneum_lcl|NC_007423.3_cds_XP_971478.1_16258'          :  "Tcastaneum",
  'Tcastaneum_lcl|NC_007423.3_cds_XP_015837921.1_16259'       :  "Tcastaneum",
  'Tcastaneum_lcl|NC_007423.3_cds_XP_015837922.1_16260'       :  "Tcastaneum",
  'Dplexippus_DPOGS211186-TA'                                 :  "Dplexippus",
  'Tcastaneum_lcl|NC_007417.3_cds_XP_015840522.1_3417'        :  "Tcastaneum",
  'Tcastaneum_lcl|NC_007417.3_cds_XP_008190275.2_3418'        :  "Tcastaneum",
  'Tcastaneum_lcl|NC_007417.3_cds_XP_972369.1_3419'           :  "Tcastaneum",
  'Aglabripennis_lcl|NW_019416354.1_cds_XP_018565491.1_5686'  :  "Aglabripennis",
  'Tannulatus2_6776066.g34'                                   :  "Tannulatus",
  'Tpilosus1_2065760.g33'                                     :  "Tpilosus",
  'Tpilosus2_4168272.g56'                                     :  "Tpilosus",
  'Tfemoratus1_9243440.g114'                                  :  "Tfemoratus",
  'Tfemoratus2_8651013.g277'                                  :  "Tfemoratus",
  'Ttetrophthalmus_maker-167874-exonerate_protein2genome-gene-0.0-mRNA-1'  :  "Ttetrophthalmus",
  'Tdiscoideus1_5565729.g65'                                  :  "Tdiscoideus",
  'Tskillmani1_13290908.g54'                                  :  "Tskillmani",
}

I could not resolve the unhashable type: 'dict_keys' error. I convert it to a tuple but got a different error.

In [324]:
for leaf in iqtree:
    leaf.order = order[leaf.name]

TypeError: unhashable type: 'dict_keys'

Part of this script works fine but I do not know how to get ride of the last part:
  
  The genera of dict_keys(['Dplexippus_DPOGS213623-TA'...

In [326]:
for leaf in iqtree:
    print(f"The genera of {leaf.name} is {leaf.order}")

The genera of Dplexippus_DPOGS213623-TA is Dplexippus
The genera of Aglabripennis_lcl|NW_019416398.1_cds_XP_018562049.1_6847 is Aglabripennis
The genera of Aglabripennis_lcl|NW_019416398.1_cds_XP_018562054.1_6850 is Aglabripennis
The genera of Aglabripennis_lcl|NW_019416398.1_cds_XP_018562052.1_6851 is Aglabripennis
The genera of Aglabripennis_lcl|NW_019416398.1_cds_XP_018562055.1_6853 is Aglabripennis
The genera of Aglabripennis_lcl|NW_019416398.1_cds_XP_018562050.1_6848 is Aglabripennis
The genera of Aglabripennis_lcl|NW_019416398.1_cds_XP_018562053.1_6852 is Aglabripennis
The genera of Aglabripennis_lcl|NW_019416398.1_cds_XP_018562056.1_6854 is Aglabripennis
The genera of Aglabripennis_lcl|NW_019416398.1_cds_XP_018562051.1_6849 is Aglabripennis
The genera of Aglabripennis_lcl|NW_019416398.1_cds_XP_018562057.1_6855 is Aglabripennis
The genera of Tfemoratus2_4327769.g89 is Tfemoratus
The genera of Tannulatus1_2465216.g23 is Tannulatus
The genera of Tpilosus2_4819112.g7 is Tpilosus
The